## Extract Poses from Amass Dataset

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
%matplotlib inline

import sys, os
import torch
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
import contextlib


from human_body_prior.tools.omni_tools import copy2cpu as c2c

os.environ['PYOPENGL_PLATFORM'] = 'egl'

### Please remember to download the following subdataset from AMASS website: https://amass.is.tue.mpg.de/download.php. Note only download the <u>SMPL+H G</u> data.
* ACCD (ACCD)
* HDM05 (MPI_HDM05)
* TCDHands (TCD_handMocap)
* SFU (SFU)
* BMLmovi (BMLmovi)
* CMU (CMU)
* Mosh (MPI_mosh)
* EKUT (EKUT)
* KIT  (KIT)
* Eyes_Janpan_Dataset (Eyes_Janpan_Dataset)
* BMLhandball (BMLhandball)
* Transitions (Transitions_mocap)
* PosePrior (MPI_Limits)
* HumanEva (HumanEva)
* SSM (SSM_synced)
* DFaust (DFaust_67)
* TotalCapture (TotalCapture)
* BMLrub (BioMotionLab_NTroje)

### Unzip all datasets. In the bracket we give the name of the unzipped file folder. Please correct yours to the given names if they are not the same.

### Place all files under the directory **.datasets/amass_data/**. The directory structure should look like the following:  
.datasets/amass_data/  
.datasets/amass_data/ACCAD/  
.datasets/amass_data/BioMotionLab_NTroje/  
.datasets/amass_data/BMLhandball/  
.datasets/amass_data/BMLmovi/   
.datasets/amass_data/CMU/  
.datasets/amass_data/DFaust_67/  
.datasets/amass_data/EKUT/  
.datasets/amass_data/Eyes_Japan_Dataset/  
.datasets/amass_data/HumanEva/  
.datasets/amass_data/KIT/  
.datasets/amass_data/MPI_HDM05/  
.datasets/amass_data/MPI_Limits/  
.datasets/amass_data/MPI_mosh/  
.datasets/amass_data/SFU/  
.datasets/amass_data/SSM_synced/  
.datasets/amass_data/TCD_handMocap/  
.datasets/amass_data/TotalCapture/  
.datasets/amass_data/Transitions_mocap/  

**Please make sure the file path are correct, otherwise it can not succeed.**

In [2]:
# Choose the device to run the body model on.
comp_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
from human_body_prior.body_model.body_model import BodyModel

male_bm_path = '../../body_models/smplh/male/model.npz'
female_bm_path = '../../body_models/smplh/female/model.npz'

num_betas = 10 # number of body parameters

male_bm = BodyModel(bm_fname=male_bm_path, num_betas=num_betas).to(comp_device)
faces = c2c(male_bm.f)

female_bm = BodyModel(bm_fname=female_bm_path, num_betas=num_betas).to(comp_device)

In [4]:
paths = []
folders = []
dataset_names = []
for root, dirs, files in os.walk('../../datasets/amass_data'):
    # print(root, dirs, files)
#     for folder in dirs:
#         folders.append(os.path.join(root, folder))
    # remove files if end with .tar.bz2 or .txt
    files = [file for file in files if not file.endswith(".tar.bz2")]
    folders.append(root)
    for name in files:
        if name.endswith(".txt"):
            continue
        dataset_name = root.split('/')[4]
        if dataset_name not in dataset_names:
            dataset_names.append(dataset_name)
        paths.append(os.path.join(root, name))

In [5]:
save_root = '../../datasets/pose_data'
save_folders = [folder.replace('../../datasets/amass_data', '../../datasets/pose_data') for folder in folders]
for folder in save_folders:
    os.makedirs(folder, exist_ok=True)

list_dataset = [
    "ACCAD",
    "BioMotionLab_NTroje",
    "BMLhandball",
    "BMLmovi",
    "CMU",
    "DFaust_67",
    "EKUT",
    "Eyes_Japan_Dataset",
    "HumanEva",
    "KIT",
    "MPI_HDM05",
    "MPI_Limits",
    "MPI_mosh",
    "SFU",
    "SSM_synced",
    "TCD_handMocap",
    "TotalCapture",
    "Transitions_mocap",
]

dataset_names = [n for n in dataset_names if n in list_dataset]
print(dataset_names)

group_path = [[path for path in paths if name in path] for name in dataset_names]

['Eyes_Japan_Dataset', 'BioMotionLab_NTroje', 'BMLhandball', 'Transitions_mocap', 'SFU', 'BMLmovi', 'MPI_HDM05', 'TotalCapture', 'MPI_Limits', 'DFaust_67', 'TCD_handMocap', 'KIT', 'MPI_mosh', 'CMU', 'SSM_synced', 'HumanEva', 'ACCAD', 'EKUT']


In [6]:
len(dataset_names), len(group_path)

(18, 18)

In [7]:
trans_matrix = np.array([[1.0, 0.0, 0.0],
                            [0.0, 0.0, 1.0],
                            [0.0, 1.0, 0.0]]) # this is to make y up
ex_fps = 20

def amass_to_pose(src_path, save_path):
    # Skip if outputs already exist
    npz_path = save_path.replace('.npy', '.npz')
    if os.path.exists(save_path) and os.path.exists(npz_path):
        # still return fps if we can read it; otherwise fall back to 0
        try:
            bdata = np.load(src_path, allow_pickle=True)
            return float(bdata.get('mocap_framerate', 0))
        except Exception:
            return 0
        
    bdata = np.load(src_path, allow_pickle=True)
    fps = 0
    
    try:
        fps = bdata['mocap_framerate']
        frame_number = bdata['trans'].shape[0]
    except:
#         print(list(bdata.keys()))
        return fps
    
    fId = 0 # frame id of the mocap sequence
    pose_seq = []
    verts_seq = []
    if bdata['gender'] == 'male':
        bm = male_bm
    else:
        bm = female_bm
    down_sample = int(fps / ex_fps)
    
#     print(frame_number)
#     print(fps)
    data_seq = {
        'root_orient': [],
        'pose_body': [],
        'pose_hand': [],
        'betas': [], 
        'transl': [],
        'gender': [], 
        'src_path': [],
        'frameId': [],
    }
    with torch.no_grad():
        count = 0
        for fId in range(0, frame_number, down_sample):
            root_orient = torch.Tensor(bdata['poses'][fId:fId+1, :3]).to(comp_device) # controls the global root orientation
            pose_body = torch.Tensor(bdata['poses'][fId:fId+1, 3:66]).to(comp_device) # controls the body
            pose_hand = torch.Tensor(bdata['poses'][fId:fId+1, 66:]).to(comp_device) # controls the finger articulation
            betas = torch.Tensor(bdata['betas'][:10][np.newaxis]).to(comp_device) # controls the body shape
            trans = torch.Tensor(bdata['trans'][fId:fId+1]).to(comp_device)
            data_seq['root_orient'].append(root_orient)
            data_seq['pose_body'].append(pose_body)
            data_seq['pose_hand'].append(pose_hand)
            data_seq['betas'].append(betas)
            data_seq['transl'].append(trans)
            data_seq['gender'].append(bdata['gender'])
            data_seq['src_path'].append(src_path)
            data_seq['frameId'].append(fId)
            # Run SMPL forward
            body = bm(pose_body=pose_body, pose_hand=pose_hand, betas=betas, root_orient=root_orient)
            verts = body.v[0]+trans
            joint_loc = body.Jtr[0] + trans
            pose_seq.append(joint_loc.unsqueeze(0))
            verts_seq.append(verts.unsqueeze(0))

            # Visualize the meshes
            # #############
            # # save body vertices as objs in output folder
            # out_folder = './meshes/'
            # os.makedirs(out_folder, exist_ok=True)
            # # save the mesh as an obj file
            # import trimesh
            # verts = torch.bmm(verts.unsqueeze(0), torch.FloatTensor(trans_matrix).unsqueeze(0))[0]
            # body_mesh = trimesh.Trimesh(vertices=c2c(verts), faces=c2c(bm.f), vertex_colors=np.tile([255, 200, 200, 255], (6890, 1)))
            # body_mesh.export(out_folder + f'body_mesh_{count:04d}.obj')
            # print(out_folder + f'body_mesh_{count:04d}.obj')
            # count += 1
            # ###############

    pose_seq = torch.cat(pose_seq, dim=0)
    pose_seq_np = pose_seq.detach().cpu().numpy()

    # Make Y up instead of Z up
    pose_seq_np_n = np.dot(pose_seq_np, trans_matrix)
    
    np.save(save_path, pose_seq_np_n)
    
    # concatenate all keys in dataseq
    for k, v in data_seq.items():
        if isinstance(v[0], torch.Tensor):
            data_seq[k] = torch.cat(v, dim=0).detach().cpu().numpy()
    np.savez(save_path.replace('.npy', '.npz'), **data_seq)
    return fps

In [8]:
group_path = group_path
all_count = sum([len(paths) for paths in group_path])
cur_count = 0

In [9]:
print(all_count)

14055


This will take a few hours for all datasets, here we take one dataset as an example

To accelerate the process, you could run multiple scripts like this at one time.

In [10]:
import time
for paths in group_path:
    dataset_name = paths[0].split('/')[4]

    pbar = tqdm(paths)
    pbar.set_description('Processing: %s'%dataset_name)
    fps = 0
    for path in pbar:
        save_path = path.replace('../../datasets/amass_data', '../../datasets/pose_data')
        save_path = save_path[:-3] + 'npy'
        fps = amass_to_pose(path, save_path)
    print(fps)
    cur_count += len(paths)
    print('Processed / All (fps %d): %d/%d'% (fps, cur_count, all_count) )
    time.sleep(0.5)
print('Done')

Processing: Eyes_Japan_Dataset: 100%|██████████| 750/750 [00:00<00:00, 1377.27it/s]


120.0
Processed / All (fps 120): 750/14055


Processing: BioMotionLab_NTroje: 100%|██████████| 3061/3061 [00:01<00:00, 1752.09it/s]


120.0
Processed / All (fps 120): 3811/14055


Processing: BMLhandball: 100%|██████████| 659/659 [00:00<00:00, 1585.17it/s]


...

120.0
Processed / All (fps 120): 13454/14055


Processing: ACCAD: 100%|██████████| 252/252 [02:13<00:00,  1.89it/s]


120.0
Processed / All (fps 120): 13706/14055


Processing: EKUT: 100%|██████████| 349/349 [01:48<00:00,  3.21it/s]


100.0
Processed / All (fps 100): 14055/14055
Done


The above code will extract poses from **AMASS** dataset, and put them under directory **"../../datasets/pose_data"**